# AI Dev Explainer - AI is useful

This is a notebook for https://www.aiexplainer.dev/useful

View all notebooks at https://github.com/chroma-core/ai_explainer

## Get an API Key from Google AI Studio
1. Go to Google AI Studio: https://aistudio.google.com
2. Click on 'Get API key' in the left sidebar
3. Create a new API key or use an existing one
4. Replace `<your_api_key>` below with your actual API key

## Run the code
In Colab - you can click, "Runtime" -> "Run All" in the topbar

In VS Code - you can click "Run All" in the topbar

In [ ]:
# install dependencies
%pip install google-genai python-dotenv

In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv

load_dotenv()

# Initialize the Anthropic client
# Make sure your GOOGLE_API_KEY is set in your .env variable
client = genai.Client()

MODEL_NAME = "gemini-2.0-flash"

def make_content(
    role: str,
    text: str,
) -> types.Content:
  """
  Create a content object for the Gemini API.
  """
  content = types.Content(
      role=role,
      parts=[
          types.Part.from_text(text=text)
      ],
  )
  return content

# Define chat helper function
def get_completion(system_prompt: str ="", user_prompt: str = ""):
  response = client.models.generate_content(
      model=MODEL_NAME,
      contents=[make_content("user", user_prompt)],
      config={
        "system_instruction": system_prompt,
        "temperature": 0.7,
      }
  )
  return response.text

In [ ]:
# The paragraph to analyze
paragraph = """Now the other princes of the Achaeans slept soundly the whole night through, but Agamemnon son of Atreus was troubled, so that he could get no rest. As when fair Hera's lord flashes his lightning in token of great rain or hail or snow when the snow-flakes whiten the ground, or again as a sign that he will open the wide jaws of hungry war, even so did Agamemnon heave many a heavy sigh, for his soul trembled within him. When he looked upon the plain of Troy he marveled at the many watchfires burning in front of Ilion... - The Iliad, Scroll 10"""

# Create the prompt for Gemini
prompt = f"""List the names of all the people mentioned in the following paragraph, separated by commas:

<PARAGRAPH>
{paragraph}
</PARAGRAPH>

Only provide the list of names, nothing else, in the format:
<FORMAT>
<NAME_1>, <NAME_2>, <NAME_3>, ...
</FORMAT>
"""

# Get the completion from Gemini
response = get_completion(
    system_prompt="You are a helpful assistant.",
    user_prompt=prompt
)
# Print the response
print(response)

That was pretty complicated! Fortunately, Google provides another module for chat interactions. Let's try that again using the chats module.

In [ ]:
chat = client.chats.create(model='gemini-1.5-flash')
response = chat.send_message(message=prompt, config={
    "temperature": 0.7,
    "candidate_count": 1,
})
try:
    print(response.candidates[0].content.parts[0].text) # type: ignore
except AttributeError:
    print("Error retrieving response")

Much better! Let's see it once more, this time with streaming enabled, and message history as well.

In [ ]:
chat = client.chats.create(model='gemini-1.5-flash', history=[])
response = chat.send_message(prompt, config={
    "temperature": 0.7,
    "candidate_count": 1,
})
try:
    followup = "Which person is Atraeus' son? How does the text describe him?"
    streaming_response = chat.send_message_stream(message=followup, config={
        "temperature": 0.7,
        "candidate_count": 1,
    })
    for chunk in streaming_response:
        print(chunk.candidates[0].content.parts[0].text, end="", flush=True) # type: ignore
except AttributeError:
    print("Error retrieving response")